In [4]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
import umap.umap_ as umap
from NFLUtils import NFLUtils
nfl_utils = NFLUtils()

### 1. Import & set main dataframe
Import dataset, set df as the original reference dataset

In [5]:
cont_cols = nfl_utils.cont_cols
track_cols = nfl_utils.track_cols


y_col = ['H_Won']

df = pd.read_csv('footballData/combined.csv', index_col=False, low_memory=False)
df = df.sort_values(by='Date')

# Create the H_Won column
df['H_Won'] = np.where(df['H_Final'] > df['V_Final'], 1.0, 0.0)

# Combine passing & rushing TD for UMAP
df['H_passing_rushing_td'] = df['H_passing_td'] + df['H_rushing_td']
df['V_passing_rushing_td'] = df['V_passing_td'] + df['V_rushing_td']

# Create the H_final_allowed, V_final_allowed columns
df['H_Final_Allowed'] = df['V_Final']
df['V_Final_Allowed'] = df['H_Final']

# Remove the performance set
test_performance_size = 200
test_performance_df = df[df.shape[0]-test_performance_size:]
df = df[:df.shape[0]-test_performance_size]
# print(f'df after perf set removed: {df.shape}')
# print(f'df perf set size {test_performance_df.shape}')

for i in range(0,len(df.columns)):
    print(f"{df.columns[i]} {df.dtypes[i]}")

Season int64
Date object
Home_Team object
H_Q1 int64
H_Q2 int64
H_Q3 int64
H_Q4 int64
H_OT int64
H_Final int64
Visitor_Team object
V_Q1 int64
V_Q2 int64
V_Q3 int64
V_Q4 int64
V_OT int64
V_Final int64
H_First_Downs int64
V_First_Downs int64
H_Rush int64
V_Rush int64
H_Yds int64
V_Yds int64
H_TDs int64
V_TDs int64
H_Cmp int64
V_Cmp int64
H_Att int64
V_Att int64
H_Yd int64
V_Yd int64
H_TD int64
V_TD int64
H_INT int64
V_INT int64
H_Sacked int64
V_Sacked int64
H_Yards int64
V_Yards int64
H_Net_Pass_Yards int64
V_Net_Pass_Yards int64
H_Total_Yards int64
V_Total_Yards int64
H_Fumbles int64
V_Fumbles int64
H_Lost int64
V_Lost int64
H_Turnovers int64
V_Turnovers int64
H_Penalties int64
V_Penalties int64
H_Yards.1 int64
V_Yards.1 int64
H_Third_Down_Conv object
V_Third_Down_Conv object
H_Fourth_Down_Conv object
V_Fourth_Down_Conv object
H_Time_of_Possession object
V_Time_of_Possession object
H_passing_att float64
H_passing_cmp float64
H_passing_int float64
H_passing_lng float64
H_passing_sk float

/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/37712009.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{df.columns[i]} {df.dtypes[i]}")
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/37712009.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{df.columns[i]} {df.dtypes[i]}")
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/37712009.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by posit

# Check and remove missing data
from https://www.kaggle.com/code/gpreda/santander-eda-and-prediction

In [6]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [7]:
pd.set_option('display.max_columns', None)
df.dropna()
missing_data(df)

,Season,Date,Home_Team,H_Q1,H_Q2,H_Q3,H_Q4,H_OT,H_Final,Visitor_Team,V_Q1,V_Q2,V_Q3,V_Q4,V_OT,V_Final,H_First_Downs,V_First_Downs,H_Rush,V_Rush,H_Yds,V_Yds,H_TDs,V_TDs,H_Cmp,V_Cmp,H_Att,V_Att,H_Yd,V_Yd,H_TD,V_TD,H_INT,V_INT,H_Sacked,V_Sacked,H_Yards,V_Yards,H_Net_Pass_Yards,V_Net_Pass_Yards,H_Total_Yards,V_Total_Yards,H_Fumbles,V_Fumbles,H_Lost,V_Lost,H_Turnovers,V_Turnovers,H_Penalties,V_Penalties,H_Yards.1,V_Yards.1,H_Third_Down_Conv,V_Third_Down_Conv,H_Fourth_Down_Conv,V_Fourth_Down_Conv,H_Time_of_Possession,V_Time_of_Possession,H_passing_att,H_passing_cmp,H_passing_int,H_passing_lng,H_passing_sk,H_passing_td,H_passing_yds,H_receiving_lng,H_receiving_td,H_receiving_yds,H_rushing_att,H_rushing_lng,H_rushing_td,H_rushing_yds,V_passing_att,V_passing_cmp,V_passing_int,V_passing_lng,V_passing_sk,V_passing_td,V_passing_yds,V_receiving_lng,V_receiving_td,V_receiving_yds,V_rushing_att,V_rushing_lng,V_rushing_td,V_rushing_yds,H_def_interceptions_int,H_def_interceptions_lng,H_def_interceptions_td,H_def_interceptions_yds,H_fumbles_ff,H_fumbles_fr,H_fumbles_td,H_fumbles_yds,H_sk,H_tackles_ast,H_tackles_comb,H_tackles_solo,V_def_interceptions_int,V_def_interceptions_lng,V_def_interceptions_td,V_def_interceptions_yds,V_fumbles_ff,V_fumbles_fr,V_fumbles_td,V_fumbles_yds,V_sk,V_tackles_ast,V_tackles_comb,V_tackles_solo,H_kick_returns_lng,H_kick_returns_rt,H_kick_returns_td,H_kick_returns_yds,H_punt_returns_lng,H_punt_returns_ret,H_punt_returns_td,H_punt_returns_yds,V_kick_returns_lng,V_kick_returns_rt,V_kick_returns_td,V_kick_returns_yds,V_punt_returns_lng,V_punt_returns_ret,V_punt_returns_td,V_punt_returns_yds,H_punting_lng,H_punting_pnt,H_punting_yds,H_scoring_fga,H_scoring_fgm,H_scoring_xpa,H_scoring_xpm,V_punting_lng,V_punting_pnt,V_punting_yds,V_scoring_fga,V_scoring_fgm,V_scoring_xpa,V_scoring_xpm,H_halftime_odds,V_halftime_odds,H_start_odds,V_start_odds,H_Won,H_passing_rushing_td,V_passing_rushing_td,H_Final_Allowed,V_Final_Allowed
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,29,31,29,27,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.397997,0.37232,0.397997,0.37232,0.346643,0.320965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Types,int64,object,object,int64,int64,int64,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64


In the above table, we see the following sets of columns that contain empty values:

In [8]:
columns_with_empty_values = df.columns[df.isnull().sum() > 0].tolist()
print(columns_with_empty_values)

['H_Third_Down_Conv', 'V_Third_Down_Conv', 'H_Fourth_Down_Conv', 'V_Fourth_Down_Conv', 'H_Time_of_Possession', 'V_Time_of_Possession']


Since some of these columns are strings, replace them with NaN and drop them via df.dropna

There are so few of them that we don't really need to worry about the missing games interfering with the overall prediction

In [9]:
for column in columns_with_empty_values:
    df[column].replace('', np.nan, inplace=True)
df.dropna(subset=columns_with_empty_values, inplace=True)
missing_data(df)

/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/2769842864.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].replace('', np.nan, inplace=True)


,Season,Date,Home_Team,H_Q1,H_Q2,H_Q3,H_Q4,H_OT,H_Final,Visitor_Team,V_Q1,V_Q2,V_Q3,V_Q4,V_OT,V_Final,H_First_Downs,V_First_Downs,H_Rush,V_Rush,H_Yds,V_Yds,H_TDs,V_TDs,H_Cmp,V_Cmp,H_Att,V_Att,H_Yd,V_Yd,H_TD,V_TD,H_INT,V_INT,H_Sacked,V_Sacked,H_Yards,V_Yards,H_Net_Pass_Yards,V_Net_Pass_Yards,H_Total_Yards,V_Total_Yards,H_Fumbles,V_Fumbles,H_Lost,V_Lost,H_Turnovers,V_Turnovers,H_Penalties,V_Penalties,H_Yards.1,V_Yards.1,H_Third_Down_Conv,V_Third_Down_Conv,H_Fourth_Down_Conv,V_Fourth_Down_Conv,H_Time_of_Possession,V_Time_of_Possession,H_passing_att,H_passing_cmp,H_passing_int,H_passing_lng,H_passing_sk,H_passing_td,H_passing_yds,H_receiving_lng,H_receiving_td,H_receiving_yds,H_rushing_att,H_rushing_lng,H_rushing_td,H_rushing_yds,V_passing_att,V_passing_cmp,V_passing_int,V_passing_lng,V_passing_sk,V_passing_td,V_passing_yds,V_receiving_lng,V_receiving_td,V_receiving_yds,V_rushing_att,V_rushing_lng,V_rushing_td,V_rushing_yds,H_def_interceptions_int,H_def_interceptions_lng,H_def_interceptions_td,H_def_interceptions_yds,H_fumbles_ff,H_fumbles_fr,H_fumbles_td,H_fumbles_yds,H_sk,H_tackles_ast,H_tackles_comb,H_tackles_solo,V_def_interceptions_int,V_def_interceptions_lng,V_def_interceptions_td,V_def_interceptions_yds,V_fumbles_ff,V_fumbles_fr,V_fumbles_td,V_fumbles_yds,V_sk,V_tackles_ast,V_tackles_comb,V_tackles_solo,H_kick_returns_lng,H_kick_returns_rt,H_kick_returns_td,H_kick_returns_yds,H_punt_returns_lng,H_punt_returns_ret,H_punt_returns_td,H_punt_returns_yds,V_kick_returns_lng,V_kick_returns_rt,V_kick_returns_td,V_kick_returns_yds,V_punt_returns_lng,V_punt_returns_ret,V_punt_returns_td,V_punt_returns_yds,H_punting_lng,H_punting_pnt,H_punting_yds,H_scoring_fga,H_scoring_fgm,H_scoring_xpa,H_scoring_xpm,V_punting_lng,V_punting_pnt,V_punting_yds,V_scoring_fga,V_scoring_fgm,V_scoring_xpa,V_scoring_xpm,H_halftime_odds,V_halftime_odds,H_start_odds,V_start_odds,H_Won,H_passing_rushing_td,V_passing_rushing_td,H_Final_Allowed,V_Final_Allowed
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Types,int64,object,object,int64,int64,int64,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64


### Convert
Time_of_Possession from %M:%S to %M
Third & fourth down conversion from attempts-total to percentage

In [10]:
for team in ["H", "V"]:
    # Time of possessiion conversion to minutes only
    df[f"{team}_Time_of_Possession"] = pd.to_datetime(df[f"{team}_Time_of_Possession"], format="%M:%S").dt.minute
    
    # Conversion from attempts-total to percentage
    df[f'{team}_Third_Down_Conv'] = df[f'{team}_Third_Down_Conv'].apply(
        lambda x: 0 if int(x.split('-')[1]) == 0 else (int(x.split('-')[0]) / int(x.split('-')[1])) * 100
    )
    df[f'{team}_Fourth_Down_Conv'] = df[f'{team}_Fourth_Down_Conv'].apply(
        lambda x: 0 if int(x.split('-')[1]) == 0 else (int(x.split('-')[0]) / int(x.split('-')[1])) * 100
    )

In [11]:
for col in track_cols[1:]:
    if col in df.columns and df[f'H_{col}'].dtype == 'object':
        print(f'converting {col} to int')
        df.astype({f'H_{col}': 'float32'})
        df.astype({f'V_{col}': 'float32'})

# Combine similar columns
Here we combine the 8 kick_returns & punt_returns columns into 4 kick_punt returns columns. We will calculate maximum between each longest punt (lng) columns and will add the rest

In [12]:
# These columns will be dropped and ignored after everything is calculated
drop_columns = [
    # Removed b/c correlation matrix similarity
    "H_passing_yds", "V_passing_yds",
    "H_receiving_yds", "V_receiving_yds",    
    "H_def_interceptions_lng", "V_def_interceptions_lng",
    "H_receiving_td", "V_receiving_td",
    
    # Removed b/c unimportant (based on domain knowledge)
    "H_punting_lng", "V_punting_lng",
    
    # Removed b/c replaced with meta-feature
    "H_punting_yds", "V_punting_yds",
    "H_scoring_fgm", "V_scoring_fgm",
    "H_scoring_xpm", "V_scoring_xpm"
]


# These columns will be added to create the new (key) column
# The columns composing the key column will be dropped
# { <New column name> : [<combine column names>, ...] }
combine_dict_add = {
    "kick_punt_returns_rt": ["kick_returns_rt", "punt_returns_ret"],
    "kick_punt_returns_td": ["kick_returns_td", "punt_returns_td"],
    "kick_punt_returns_yds": ["kick_returns_yds", "punt_returns_yds"]
}

# These columns will be divided. None will be dropped unless in drop_columns
# { <New column name> : [<numerator column name>, <denominator column name>] }
combine_dict_divide = {
    "scoring_fgp": ["scoring_fgm", "scoring_fga"],
    "scoring_xpp": ["scoring_xpm", "scoring_xpa"],
    "punting_avg": ["punting_yds", "punting_pnt"],
}

# Perform column based operations (max, add, divide)
for team in ["H", "V"]:
    # For the longest kick/punt, get the max & drop columns
    df[f'{team}_kick_punt_returns_lng'] = np.maximum(df[f'{team}_kick_returns_lng'], df[f'{team}_punt_returns_lng'])
    df.drop([f'{team}_kick_returns_lng', f'{team}_punt_returns_lng'], axis=1, inplace=True)

    # -------- Columns to add --------
    for (key, value) in combine_dict_add.items():
        df[f'{team}_{key}'] = df[f'{team}_{value[0]}'] + df[f'{team}_{value[1]}']
        df.drop([f'{team}_{value[0]}', f'{team}_{value[1]}'], axis=1, inplace=True)

    # -------- Columns to divide --------
    for (key, value) in combine_dict_divide.items():
        df[f'{team}_{key}'] = np.where(df[f'{team}_{value[1]}'] > 0, df[f'{team}_{value[0]}']/df[f'{team}_{value[1]}'], 0)

        
# Drop columns
df.drop(drop_columns, axis=1, inplace=True)
print(len(df.columns))
print(list(df.columns))

131
['Season', 'Date', 'Home_Team', 'H_Q1', 'H_Q2', 'H_Q3', 'H_Q4', 'H_OT', 'H_Final', 'Visitor_Team', 'V_Q1', 'V_Q2', 'V_Q3', 'V_Q4', 'V_OT', 'V_Final', 'H_First_Downs', 'V_First_Downs', 'H_Rush', 'V_Rush', 'H_Yds', 'V_Yds', 'H_TDs', 'V_TDs', 'H_Cmp', 'V_Cmp', 'H_Att', 'V_Att', 'H_Yd', 'V_Yd', 'H_TD', 'V_TD', 'H_INT', 'V_INT', 'H_Sacked', 'V_Sacked', 'H_Yards', 'V_Yards', 'H_Net_Pass_Yards', 'V_Net_Pass_Yards', 'H_Total_Yards', 'V_Total_Yards', 'H_Fumbles', 'V_Fumbles', 'H_Lost', 'V_Lost', 'H_Turnovers', 'V_Turnovers', 'H_Penalties', 'V_Penalties', 'H_Yards.1', 'V_Yards.1', 'H_Third_Down_Conv', 'V_Third_Down_Conv', 'H_Fourth_Down_Conv', 'V_Fourth_Down_Conv', 'H_Time_of_Possession', 'V_Time_of_Possession', 'H_passing_att', 'H_passing_cmp', 'H_passing_int', 'H_passing_lng', 'H_passing_sk', 'H_passing_td', 'H_receiving_lng', 'H_rushing_att', 'H_rushing_lng', 'H_rushing_td', 'H_rushing_yds', 'V_passing_att', 'V_passing_cmp', 'V_passing_int', 'V_passing_lng', 'V_passing_sk', 'V_passing_td'

In [13]:
# df['Date'].values
df['Season'].values

array([1995, 1995, 1995, ..., 2024, 2024, 2024])

### 2. Create a dict to track the track_cols array
create another dict to track previous games for each team during the year

In [14]:
track_dict = {}

for row in df.itertuples():
    # year = row.Date.split('-')[0]
    year = row.Season
    home_team = row.Home_Team
    visitor_team = row.Visitor_Team
    
    # Home or visitor team has < minimum_window total games
    for col in track_cols:
        home_column_name = f'{year}_{home_team}_{col}'
        visitor_column_name = f'{year}_{visitor_team}_{col}'
        
        # Home team
        home_col = col if col == 'Date' else 'H_' + col
        if home_column_name in track_dict:
            track_dict[home_column_name].append(getattr(row, home_col))
        else:
            track_dict[home_column_name] = [getattr(row, home_col)]
        
        # Visitor team
        visitor_col = col if col == 'Date' else 'V_' + col
        if visitor_column_name in track_dict:
            track_dict[visitor_column_name].append(getattr(row, visitor_col))
        else:
            track_dict[visitor_column_name] = [getattr(row, visitor_col)]

In [15]:
track_dict['1999_NWE_Rush']

[27, 121, 57, 35, 19, 122, 133, 43, 188, 19, 25, 63, 20, 27, 168, 119]

### 3. Use track_dict to enforce minimum_window, update df or drop row


In [16]:
print(df.shape)
# for index in df.itertuples():
#    print(f"{index.Index} {index.Date} out of {df.shape[0]}")
   

(7757, 131)


In [17]:
minimum_window = 4
ema_span = 7
print(df.shape)

indices_to_drop = []
current_count = 0

for row in df.itertuples():
    if current_count % 300 == 0:
        print(f'{current_count}/{df.shape[0]}')
    current_count = current_count + 1
    index = row.Index
    # year = row.Date.split('-')[0]
    year = row.Season
    home_team = row.Home_Team
    visitor_team = row.Visitor_Team
    # Home team min window
    home_date_column = f'{year}_{home_team}_Date'
    visitor_date_column = f'{year}_{visitor_team}_Date'

    # Current row is older than Home team at min_window
    if len(track_dict[home_date_column]) > minimum_window and row.Date <= track_dict[home_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue
    # Current row is older than Visitor team at min_window
    if len(track_dict[visitor_date_column]) > minimum_window and row.Date <= track_dict[visitor_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue

    home_date_index = track_dict[home_date_column].index(row.Date)
    visitor_date_index = track_dict[visitor_date_column].index(row.Date)
    # print(f'H: {home_date_index} V: {visitor_date_index}')

    # Update df to have average for each track_cols (Ignoring 'Date', 'datediff' the 1-2nd item)
    for col in track_cols[1:]:
        if col in ['start_odds', 'halftime_odds']:
            continue
        else:
            # Update df to have average for each track_cols (Ignoring 'Date', 'datediff' the 1-2nd item)
            # Update home
            home_col_list = track_dict[f'{year}_{home_team}_{col}'][:home_date_index-1]
            dataframe_val = pd.DataFrame({'value': home_col_list})
            ema = dataframe_val['value'].ewm(span=min(ema_span, len(home_col_list)), adjust=True).mean().iloc[-1]
            df.at[index, 'H_' + col] = ema

            # Update Visitor
            visitor_col_list = track_dict[f'{year}_{visitor_team}_{col}'][:visitor_date_index-1]
            dataframe_val = pd.DataFrame({'value': visitor_col_list})
            ema = dataframe_val['value'].ewm(span=min(minimum_window, len(visitor_col_list)), adjust=False).mean().iloc[-1]
            df.at[index, 'V_' + col] = ema


    # --------------------------------------------------- 
    # ------------------ Custom Columns -----------------
    # --------------------------------------------------- 
    #
    
    # 1. Add variant of Bill James pythagorean expectation (NFL).
    # Recent games weighted more heavily since 'Final' columns not excluded from the above loop
    home_points_for = sum(track_dict[f'{year}_{home_team}_Final'][:home_date_index-1])
    home_points_against = sum(track_dict[f'{year}_{home_team}_Final_Allowed'][:home_date_index-1])
    df.at[index, 'H_pythagorean'] = home_points_for**2.37 / (home_points_for**2.37 + home_points_against**2.37)

    visitor_points_for = sum(track_dict[f'{year}_{visitor_team}_Final'][:visitor_date_index-1])
    visitor_points_against = sum(track_dict[f'{year}_{visitor_team}_Final_Allowed'][:visitor_date_index-1])
    df.at[index, 'V_pythagorean'] = visitor_points_for**2.37 / (visitor_points_for**2.37 + visitor_points_against**2.37)            
            
        
    # 2. Add num days since last game for home, visitor
    df.at[index, f'H_datediff'] = 0
    if home_date_index > 0:
        current_game_date = datetime.strptime(track_dict[home_date_column][home_date_index], "%Y-%m-%d")
        previous_game_date = datetime.strptime(track_dict[home_date_column][home_date_index-1], "%Y-%m-%d")
        game_diff = int((current_game_date - previous_game_date).days)
        # print(f'{current_game_date} minus {previous_game_date} is {game_diff}')
        df.at[index, f'H_datediff'] = game_diff
    
    df.at[index, f'V_datediff'] = 0
    if visitor_date_index > 0:
        current_game_date = datetime.strptime(track_dict[visitor_date_column][visitor_date_index], "%Y-%m-%d")
        previous_game_date = datetime.strptime(track_dict[visitor_date_column][visitor_date_index-1], "%Y-%m-%d")
        game_diff = int((current_game_date - previous_game_date).days)
        # print(f'{current_game_date} minus {previous_game_date} is {game_diff}')
        df.at[index, f'V_datediff'] = game_diff
        
df.drop(indices_to_drop, inplace=True)

# Add custom metrics to track_cols so it creates the difference (D_) column
track_cols.append('datediff')
track_cols.append('pythagorean')
for col in track_cols[1:]:
    cont_cols.append('D_' + col)
    df['D_' + col] = (df['H_' + col] - df['V_' + col]).round(3) # Round to 3 sig figs

print(track_cols)
track_cols.pop()


print(df.shape)

(7757, 131)
0/7757


/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/4174120937.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '18.753676470588232' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'H_' + col] = ema
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/4174120937.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '16.296' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'V_' + col] = ema
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_21064/4174120937.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '91.2720588235294' has dtype incompatible with int64, please explicitly cast to a compatible dtyp

300/7757
600/7757


KeyboardInterrupt: 

dataframe_val
   value
0      1
1     23
2      0
3     18
4      4
5      1
ema
0     1.000000
1    13.833333
2     7.614679
3    11.626126
4     8.949569
5     6.330412
Name: value, dtype: float64
ema is: 6.330412451972085
dataframe_val
   value
0     26
1    260
2     39
3    189
4     21
5     41
ema
0     26.000000
1    162.500000
2    106.981651
3    138.662162
4     97.366028
5     78.795029
Name: value, dtype: float64
ema is: 78.79502862071669
dataframe_val
   value
0      1
1      1
2      3
3      1
4      0
5      2
ema
0    1.000000
1    1.000000
2    1.899083
3    1.551802
4    1.007163
5    1.334274
Name: value, dtype: float64
ema is: 1.3342742884027288
dataframe_val
   value
0      1
1      0
2      1
3      5
4      3
5      3
ema
0    1.000000
1    0.416667
2    0.678899
3    2.347973
4    2.576816
5    2.716243
Name: value, dtype: float64
ema is: 2.7162432368854383
dataframe_val
   value
0      5
1      9
2      2
3     13
4      1
5      1
ema
0    5.000000
1    7.33

...

ema
0    0.000000
1    1.142857
2    0.648649
3    0.777143
4    0.522407
5    1.275913
6    1.196309
7    0.863959
8    0.900730
9    1.456866
Name: value, dtype: float64
ema is: 1.4568657550526665
dataframe_val
   value
0  111.0
1  168.0
2  138.0
3  107.0
4  155.0
5  272.0
6  153.0
7  114.0
8  110.0
9  125.0
ema
0    111.000000
1    143.571429
2    141.162162
3    128.668571
4    137.299616
5    178.265815
6    170.976333
7    155.147588
8    142.944420
9    138.190611
Name: value, dtype: float64
ema is: 138.1906112718501
dataframe_val
   value
0    0.0
1    3.0
2    3.0
3    1.0
4    0.0
5    4.0
6    1.0
7    1.0
8    1.0
9    4.0
ema
0    0.000000
1    1.714286
2    2.270270
3    1.805714
4    1.213828
5    2.061182
6    1.755019
7    1.545265
8    1.397883
9    2.087232
Name: value, dtype: float64
ema is: 2.087231576298575
dataframe_val
   value
0    1.0
1    2.0
2    0.0
3    2.0
4    1.0
5    1.0
6    3.0
7    2.0
8    1.0
9    0.0
ema
0    1.000000
1    1.571429
2    0.891892


dataframe_val
    value
0   108.0
1   125.0
2    45.0
3    97.0
4    77.0
5    79.0
6   110.0
7    77.0
8   180.0
9    71.0
10  102.0
ema
0     108.000000
1     117.714286
2      86.270270
3      90.194286
4      85.869398
5      83.780220
6      91.344932
7      87.359729
8     112.399932
9     101.432325
10    101.580502
Name: value, dtype: float64
ema is: 101.58050183251488
dataframe_val
    value
0     5.0
1     6.0
2     8.0
3     2.0
4     3.0
5     1.0
6     8.0
7     6.0
8     6.0
9     4.0
10    5.0
ema
0     5.000000
1     5.571429
2     6.621622
3     4.931429
4     4.298335
5     3.295218
6     4.652603
7     5.026927
8     5.289944
9     4.948214
10    4.961731
Name: value, dtype: float64
ema is: 4.961731144687649
dataframe_val
        value
0   43.800000
1   39.333333
2   47.125000
3   34.500000
4   36.333333
5   40.000000
6   40.875000
7   41.000000
8   42.166667
9   48.750000
10  43.800000
ema
0     43.800000
1     41.247619
2     43.789189
3     40.392000
4     39.0616

ema
0     2.000000
1     2.000000
2     1.567568
3     2.091429
4     1.733675
5     2.118800
6     2.084525
7     1.783230
8     2.382412
9     2.546022
10    2.403497
Name: value, dtype: float64
ema is: 2.4034972992093664
dataframe_val
    value
0     2.0
1     0.0
2     3.0
3     0.0
4     2.0
5     2.0
6     2.0
7     1.0
8     0.0
9     0.0
10    2.0
ema
0     2.000000
1     0.857143
2     1.783784
3     1.131429
4     1.416133
5     1.593704
6     1.710925
7     1.513421
8     1.104351
9     0.811788
10    1.121940
Name: value, dtype: float64
ema is: 1.1219402179202853
dataframe_val
    value
0   127.0
1   131.0
2   132.0
3   100.0
4   134.0
5   114.0
6   192.0
7    95.0
8   136.0
9   106.0
10  143.0
ema
0     127.000000
1     129.285714
2     130.459459
3     119.320000
4     124.131882
5     121.050490
6     141.520251
7     128.596337
8     130.597511
9     124.081175
10    129.019450
Name: value, dtype: float64
ema is: 129.01945007377108
dataframe_val
    value
0    40.0
1   

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
Name: value, dtype: float64
ema is: 0.0
dataframe_val
    value
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
ema
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
Name: value, dtype: float64
ema is: 0.0
dataframe_val
    value
0     6.0
1     6.0
2     7.0
3     8.0
4     6.0
5     8.0
6     7.0
7     4.0
8     7.0
9     5.0
10    4.0
ema
0     6.000000
1     6.000000
2     6.432432
3     7.005714
4     6.676056
5     7.078705
6     7.055998
7     6.207003
8     6.421346
9     6.044805
10    5.511061
Name: value, dtype: float64
ema is: 5.511061180830124
dataframe_val
    value
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
ema
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0

KeyboardInterrupt: 

# UMAP

In [15]:
# df["D_passing_rushing_td"].head()
print((df["D_kick_punt_returns_td"] > 0).sum())
print((df["D_kick_punt_returns_td"] == 0).sum())

1422
2593


In [16]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler


# Explosive play indicators
return_game_explosives = ["D_kick_punt_returns_lng", "D_kick_punt_returns_rt", "D_kick_punt_returns_yds"]
return_game_df = df[return_game_explosives]
scaled_return_game_df = StandardScaler().fit_transform(return_game_df)

reducer = umap.UMAP(random_state=42, n_neighbors=40, min_dist=0.1)
embedding = reducer.fit_transform(scaled_return_game_df)
print(embedding.shape)

# Create the two new columns, drop the 4
df['kick_punt_umap_dim_1'] = embedding[:,0]
df['kick_punt_umap_dim_2'] = embedding[:,1]
df.drop(return_game_explosives + ['D_kick_punt_returns_td'], axis=1, inplace=True)
# Save UMAP model as pickle




# ------------- UMAP explore parameters -------------
# start_neighbors = 5
#while start_neighbors < 41:
#    start_dist = 0.1
#    while start_dist < 0.5:
#        reducer = umap.UMAP(random_state=42, n_neighbors=start_neighbors, min_dist=0.1)
#        embedding = reducer.fit_transform(scaled_return_game_df)
#        # Create a new figure for each plot
#        plt.scatter(
#            embedding[:, 0],
#            embedding[:, 1],
#            #c=df["D_kick_punt_returns_td"].astype(float),
#            #c=df["H_Won"],
#            c=df["D_Final"],            
#            cmap='Spectral',
#            s=5
#        )
#        plt.gca().set_aspect('equal', 'datalim')
#        plt.title(f'UMAP with n_neighbors={start_neighbors} start_dist={start_dist}')
#        plt.show()
#        start_dist = start_dist * 2
#    start_neighbors = start_neighbors * 2


/Users/forbesjon2/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(5308, 2)


/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_17373/3859661322.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['kick_punt_umap_dim_1'] = embedding[:,0]
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_17373/3859661322.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['kick_punt_umap_dim_2'] = embedding[:,1]


In [17]:
df.to_csv(f'./footballData/CombinedSlidingWindow{minimum_window}.csv')

In [18]:
print(len(cont_cols))

173
